## Load data

In [1]:
import numpy as np
from keras.utils.np_utils import to_categorical
import keras

ImportError: No module named 'keras'

In [6]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
datapath = '/media/windows-share/EEGs_Guinea-Bissau_np/' #'/media/sf_VBox_Shared/timeseries/EEGs_Guinea-Bissau_np/'

In [8]:
condition = '_10seconds_closed.npy'
X_train = np.load(datapath+'X_train'+condition)
y_train = np.load(datapath+'y_train'+condition)
X_val = np.load(datapath+'X_valid'+condition)
y_val = np.load(datapath+'y_valid'+condition)
X_test = np.load(datapath+'X_test'+condition)
y_test = np.load(datapath+'y_test'+condition)

In [9]:
classlabels = list(set(y_train))
mapclasses = {classlabels[i] : i for i in range(len(classlabels))}
print(mapclasses)

{u'Control': 0, u'Epilepsy': 1}


In [10]:
y_train = np.array([mapclasses[c] for c in y_train], dtype='int')
y_val = np.array([mapclasses[c] for c in y_test], dtype='int')
y_test = np.array([mapclasses[c] for c in y_test], dtype='int')
y_train_binary = to_categorical(y_train)
y_val_binary = to_categorical(y_val)
y_test_binary = to_categorical(y_test)

In [11]:
y_val_binary

array([[ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 0.,  1.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.]])

In [12]:
print(X_train.shape)

(108, 1280, 14)


In [13]:
num_classes = y_train_binary.shape[1]
print(num_classes)

2


## Generate models

In [14]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Convolution1D, Flatten, MaxPooling1D
from keras.optimizers import Adam

In [15]:
import sys
import os
import numpy as np
sys.path.insert(0, os.path.abspath('..'))

from mcfly import modelgen, find_architecture

In [16]:
np.random.seed = 321

In [17]:
%%time
models = modelgen.generate_models(X_train.shape,
                                  num_classes,
                                  number_of_models = 3,
                                  model_type = 'CNN',
                                  cnn_min_layers=2,
                                  cnn_max_layers=3,
                                 low_lr=2, high_lr=8)

CPU times: user 1.41 s, sys: 360 ms, total: 1.77 s
Wall time: 2.79 s


In [18]:
%%time
for model, params, model_types in models:
    print(params)
    model.summary()

{'fc_hidden_nodes': 85, 'learning_rate': 4.9731513645518736e-06, 'regularization_rate': 0.02138795781656715, 'filters': array([12, 63])}
____________________________________________________________________________________________________
Layer (type)                       Output Shape        Param #     Connected to                     
batchnormalization_1 (BatchNormaliz(None, 1280, 14)    28          batchnormalization_input_1[0][0] 
____________________________________________________________________________________________________
convolution1d_1 (Convolution1D)    (None, 1280, 12)    516         batchnormalization_1[0][0]       
____________________________________________________________________________________________________
batchnormalization_2 (BatchNormaliz(None, 1280, 12)    24          convolution1d_1[0][0]            
____________________________________________________________________________________________________
activation_1 (Activation)          (None, 1280, 12)    

## Compare models

In [20]:
%%time
histories, val_accuracies, val_losses = find_architecture.train_models_on_samples(X_train, y_train_binary,
                                                                                 X_val, y_val_binary,
                                                                                 models,nr_epochs=2,
                                                                                  subset_size=10,
                                                                                  verbose=True,
                                                                                 outputpath='/home/vincent/estep/')

Train on 10 samples, validate on 20 samples
Epoch 1/2
10/10 [==============================] - 0s - loss: 3.7879 - acc: 1.0000 - val_loss: 8.8650 - val_acc: 0.4500
Epoch 2/2
10/10 [==============================] - 0s - loss: 3.7153 - acc: 1.0000 - val_loss: 8.8650 - val_acc: 0.4500
{'fc_hidden_nodes': 85, 'train_loss': [3.7878670692443848, 3.7153491973876953], 'train_acc': [1.0, 1.0], 'learning_rate': 4.9731513645518736e-06, 'modeltype': 'CNN', 'regularization_rate': 0.02138795781656715, 'val_acc': [0.44999998807907104, 0.44999998807907104], 'filters': [12, 63], 'val_loss': [8.8649530410766602, 8.8649530410766602]}
Train on 10 samples, validate on 20 samples
Epoch 1/2
10/10 [==============================] - 0s - loss: 0.7569 - acc: 1.0000 - val_loss: 4.5119 - val_acc: 0.5500
Epoch 2/2
10/10 [==============================] - 0s - loss: 0.7345 - acc: 1.0000 - val_loss: 7.2531 - val_acc: 0.5500
{'fc_hidden_nodes': 82, 'train_loss': [0.75686061382293701, 0.73449933528900146], 'train_acc

In [ ]:
models

In [ ]:
def storetrainhist2json(models,histories,outputpath):
    import json
    para = {'params':[str(params) for model, params, model_types in models]}
    modelty = {'modeltype':[str(model_types) for model, params, model_types in models]}
    histo = {'train_acc': [history.history['acc'] for history in histories],
    'train_loss': [history.history['loss'] for history in histories],
    'val_acc': [history.history['val_acc'] for history in histories],
    'val_loss': [history.history['val_loss'] for history in histories]}
    Nmodels = len(para['params'])
    for i in range(Nmodels):
        dict = eval(para['params'][i])
        dict['modeltype'] = modelty['modeltype'][i]
        dict['train_acc'] = histo['train_acc'][i]
        dict['train_loss'] = histo['train_loss'][i]
        dict['val_acc'] = histo['val_acc'][i]
        dict['val_loss'] = histo['val_loss'][i]
        dict['filters'] = dict['filters'].tolist()
        with open(outputpath + '/model' + str(i) + '.json', 'w') as outfile:
            json.dump(dict, outfile, sort_keys = True, indent = 4,ensure_ascii=False)


In [ ]:
from keras import backend as K

# with a Sequential model
get_dens_layer_output = K.function([best_model_copy.layers[0].input, K.learning_phase()],
                                  [best_model_copy.layers[0].output])
layer_output = get_dens_layer_output([X_val, 0])[0]

In [ ]:
layer_output.shape

In [ ]:
layer_output.mean(axis=(0,1))

In [ ]:
layer = best_model.layers[0]
for w in layer.get_weights():
    print(w.shape)

## See if we can overfit on a small train set

In [ ]:
params = models[0][1]
print(params)
small_model = modelgen.generate_CNN_model(X_train.shape, num_classes, params['filters'], params['fc_hidden_nodes'],
                                  0.01, #params['learning_rate'], 
                                        regularization_rate=0)
small_model.summary()

In [ ]:
small_model.evaluate(X_val, y_val_binary)

In [ ]:
nr_epochs = 100
datasize = 20
history = small_model.fit(X_train[:datasize,:,:], y_train_binary[:datasize,:],
              nb_epoch=nr_epochs, validation_data=(X_val, y_val_binary), batch_size=10)

## Test on Testset

In [ ]:
score_test = best_model.evaluate(X_test, y_test_binary, verbose=False)
print('Score of best model: ' + str(score_test))

In [ ]:
best_model.get_config()[0]